# Crop Area Calculation

## Setup

In [1]:
# import ee
# ee.Authenticate()

In [2]:
import sys
from os.path import join, expanduser

sys.path.append(join(expanduser("~"), "Repos", "GEE_Zonal", "src"))

# Data
import geopandas as gpd
import pandas as pd

## Earth Engine APIs
import ee
from gee_zonal import ZonalStats, gpd_to_gee
from geemap.conversion import *

import requests

from plotnine import *


sys.path.append(join(expanduser("~"), "Repos", "phenolopy", "scripts"))

sys.path.append(join(expanduser("~"), "Repos", "GOSTrocks", "src"))

## Data

### Crop Mask

In [3]:
dea = ee.ImageCollection(
    "projects/sat-io/open-datasets/DEAF/CROPLAND-EXTENT/mask"
).mosaic()  # filtered
crop_data = dea.select("b1").rename("crop")

### Admin

In [5]:
adm1_ner = gpd.read_file(
    "../../data/shapefiles/ner_adm_ignn_20230720_ab_shp/NER_admbnda_adm1_IGNN_20230720.shp"
)

In [4]:
iso3 = "NER"
country = "Niger"
release_type = "gbOpen"

adm = "ADM0"
geo_url = f"https://www.geoboundaries.org/api/current/{release_type}/{iso3}/{adm}/"
res = requests.get(geo_url).json()
print("Reading " + res["gjDownloadURL"])
adm0_ner = gpd.read_file(res["gjDownloadURL"])

adm = "ADM1"
geo_url = f"https://www.geoboundaries.org/api/current/{release_type}/{iso3}/{adm}/"
res = requests.get(geo_url).json()
print("Reading " + res["gjDownloadURL"])
adm1_ner = gpd.read_file(res["gjDownloadURL"])

adm = "ADM2"
geo_url = f"https://www.geoboundaries.org/api/current/{release_type}/{iso3}/{adm}/"
res = requests.get(geo_url).json()
print("Reading " + res["gjDownloadURL"])
adm2_ner = gpd.read_file(res["gjDownloadURL"])

# adm1_ner.to_file('adm1_ner.geojson', driver='GeoJSON')
# adm1_ner = gpd.read_file('adm1_ner.geojson')
# adm2_ner.to_file('adm2_ner.geojson', driver='GeoJSON')
# adm2_ner = gpd.read_file('adm2_ner.geojson')

Reading https://github.com/wmgeolab/geoBoundaries/raw/9469f09/releaseData/gbOpen/NER/ADM0/geoBoundaries-NER-ADM0.geojson
Reading https://github.com/wmgeolab/geoBoundaries/raw/9469f09/releaseData/gbOpen/NER/ADM1/geoBoundaries-NER-ADM1.geojson
Reading https://github.com/wmgeolab/geoBoundaries/raw/9469f09/releaseData/gbOpen/NER/ADM2/geoBoundaries-NER-ADM2.geojson


In [6]:
adm1_ner.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
aoi = gpd_to_gee(adm1_ner)

In [8]:
bool_dict = {
    "0": "non_crop",
    "1": "crop",
}
zs = ZonalStats(
    target_features=aoi,
    statistic_type="count",
    scale=10,
    ee_dataset=crop_data,
    output_name="adm1_crop_count_v2",
    output_dir="niger",
)

In [9]:
zs.runZonalStats()

In [12]:
zs.reportRunTime()

Completed
Runtime: 3 minutes and 23 seconds


In [ ]:
histograms = res["histogram"]
histograms_dfs = [
    pd.DataFrame(index=[idx], data=hist_dict) for idx, hist_dict in histograms.items()
]
histogram_df = pd.concat(histograms_dfs, axis=0).fillna(0)
histogram_df.rename(columns=bool_dict, inplace=True)